In [34]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 337 kB in 1s (234 kB/s)
Reading package lists... Done


In [35]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [36]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [39]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [40]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, year
# Add the 'year' column to the DataFrame
df = df.withColumn('year', year('date'))

# Calculate the average price for 4-bedroom houses by year

result = spark.sql("SELECT year, ROUND(AVG(price), 2) AS avg_price_4bedroom FROM home_sales WHERE bedrooms = 4 GROUP BY year ORDER BY year")
# Show the result
result.show()








+----+------------------+
|year|avg_price_4bedroom|
+----+------------------+
|2019|          300263.7|
|2020|         298353.78|
|2021|         301819.44|
|2022|         296363.88|
+----+------------------+



In [41]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

result2 = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS avg_price_3bedroom FROM home_sales WHERE bedrooms = 3 AND bathrooms=3 GROUP BY date_built ORDER BY date_built")
# Show the result
result2.show()

+----------+------------------+
|date_built|avg_price_3bedroom|
+----------+------------------+
|      2010|         292859.62|
|      2011|         291117.47|
|      2012|         293683.19|
|      2013|         295962.27|
|      2014|         290852.27|
|      2015|          288770.3|
|      2016|         290555.07|
|      2017|         292676.79|
+----------+------------------+



In [42]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

result3 = spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price_3bedroom
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built ORDER BY date_built
""")

result3.show()


+----------+------------------+
|date_built|avg_price_3bedroom|
+----------+------------------+
|      2010|         285010.22|
|      2011|         276553.81|
|      2012|         307539.97|
|      2013|         303676.79|
|      2014|         298264.72|
|      2015|         297609.97|
|      2016|          293965.1|
|      2017|         280317.58|
+----------+------------------+



In [43]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

result4 = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_view
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view ORDER BY view
""")
result4.show()
start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------+
|view|avg_price_view|
+----+--------------+
|   0|     403848.51|
|   1|     401044.25|
|  10|     401868.43|
| 100|     1026669.5|
|  11|     399548.12|
|  12|     401501.32|
|  13|     398917.98|
|  14|     398570.03|
|  15|      404673.3|
|  16|     399586.53|
|  17|     398474.49|
|  18|     399332.91|
|  19|     398953.17|
|   2|     397389.25|
|  20|     399522.81|
|  21|     399758.88|
|  22|     402022.68|
|  23|     403411.92|
|  24|     400284.92|
|  25|     401298.69|
+----+--------------+
only showing top 20 rows

--- 0.00011491775512695312 seconds ---


In [44]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [45]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [46]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

result5 = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_view
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view ORDER BY view
""")
result5.show()



start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------+
|view|avg_price_view|
+----+--------------+
|   0|     403848.51|
|   1|     401044.25|
|  10|     401868.43|
| 100|     1026669.5|
|  11|     399548.12|
|  12|     401501.32|
|  13|     398917.98|
|  14|     398570.03|
|  15|      404673.3|
|  16|     399586.53|
|  17|     398474.49|
|  18|     399332.91|
|  19|     398953.17|
|   2|     397389.25|
|  20|     399522.81|
|  21|     399758.88|
|  22|     402022.68|
|  23|     403411.92|
|  24|     400284.92|
|  25|     401298.69|
+----+--------------+
only showing top 20 rows

--- 0.00010704994201660156 seconds ---


In [48]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Create or get the SparkSession
spark = SparkSession.builder.appName("CreateFormattedParquetData").getOrCreate()

# Read the data from the temporary view "home_sales" into a DataFrame
home_sales_df = spark.table("home_sales")

# Save the DataFrame partitioned by date_built to Parquet format with partitioning
home_sales_df.write.mode("overwrite").partitionBy("date_built").parquet("../Challenge Module 22/Starter_Code/home_sales_parquet_df")

In [49]:
# 11. Read the parquet formatted data.

# Create or get the SparkSession
spark = SparkSession.builder.appName("ReadParquetData").getOrCreate()

# Read the Parquet data from the specified path
parquet_path = "../Challenge Module 22/Starter_Code/home_sales_parquet_df"
parquet_df = spark.read.parquet(parquet_path)

# display parquet data
parquet_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|2021|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|2020|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|2019|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|2019|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|2021|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|

In [50]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_home_sales')

In [51]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

result6 = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_view
    FROM parquet_home_sales
    WHERE price >= 350000
    GROUP BY view ORDER BY view
""")
result6.show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------+
|view|avg_price_view|
+----+--------------+
|   0|     403848.51|
|   1|     401044.25|
|  10|     401868.43|
| 100|     1026669.5|
|  11|     399548.12|
|  12|     401501.32|
|  13|     398917.98|
|  14|     398570.03|
|  15|      404673.3|
|  16|     399586.53|
|  17|     398474.49|
|  18|     399332.91|
|  19|     398953.17|
|   2|     397389.25|
|  20|     399522.81|
|  21|     399758.88|
|  22|     402022.68|
|  23|     403411.92|
|  24|     400284.92|
|  25|     401298.69|
+----+--------------+
only showing top 20 rows

--- 0.0001087188720703125 seconds ---


In [52]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [53]:
# 15. Check if the home_sales is no longer cached

home_sales_is_cached = spark.catalog.isCached("home_sales")
print(home_sales_is_cached)

False
